In [11]:
import PyPDF2
pdf = PyPDF2.PdfReader(open('Data_For_Medical_Bot.pdf', 'rb'))

text = ""
for page in pdf.pages:
    text += page.extract_text()

In [ ]:
elements = [
    cl.Image(name="image", display="inline", path="pic.jpg")
    ]
    # Inform the user that processing has started
msg = cl.Message(content=f"Processing `{file.name}`...",elements=elements)
await msg.send()

In [17]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1200, chunk_overlap=50)
texts = text_splitter.split_text(text)

In [18]:
metadatas = [{"source": f"{i}-pl"} for i in range(len(texts))]

In [29]:
import chainlit as cl
from langchain_community.vectorstores import Chroma
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings


In [ ]:
embeddings = OllamaEmbeddings(model="nomic-embed-text")
docsearch = await cl.make_async(Chroma.from_texts)(
    texts, embeddings, metadatas=metadatas
)

In [ ]:
from langchain.memory import ChatMessageHistory, ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

message_history = ChatMessageHistory()

memory = ConversationBufferMemory(
    memory_key="chat_history",
    output_key="answer",
    chat_memory=message_history,
    return_messages=True,
)

chain = ConversationalRetrievalChain.from_llm(
    ChatOllama(model="gemma:7b"),
    chain_type="stuff",
    retriever=docsearch.as_retriever(),
    memory=memory,
    return_source_documents=True,
)

In [ ]:
msg.content = f"Processing `{pdf.name}` done. You can now ask questions!"
await msg.update()
#store the chain in user session
cl.user_session.set("chain", chain)